<h4> Please expand the respective sections to find the functions for that step along with some test cases that we tried. </h4>

### Step-0: Converting input formula into list format

Example Conversion:

∀x (p(x) ⊃ ∀y (q(y) ⊃ r(y))) becomes ['forall', 'x', ['implies', ['p', 'x'], ['forall', 'y', ['implies', ['q', 'y'], ['r', 'y']]]]]

Binary operators are given as [operator, operand1, operand2] format and unary operators are given as [operator, operand] format

Quantifiers are given as [quatifier, variable, formula] format

Predicates and functions are given as [predicate/function, variable1, variable2, ...] format

In [60]:
import re

def parse_logical_expression(formula):
    op_map = {'∀': 'forall', '∃': 'exists', '¬': 'not', '∧': 'and', '∨': 'or', '⊃': 'implies', '↔': 'iff', '=': 'equals'}
    precedence = {'=': 6, '¬': 5, '∧': 4, '∨': 3, '⊃': 2, '↔': 1, '∀': -1, '∃': -1}

    formula = re.sub(r'([∀∃¬∧∨⊃↔=(),])', r' \1 ', formula)
    tokens = [token for token in re.split(r'\s+', formula) if token]

    def next_token():
        if tokens:
            return tokens.pop(0)
        return None

    def parse_primary():
        token = next_token()
        if token in ['∀', '∃']:
            var_token = next_token()
            expr = parse_expression()
            return [op_map[token], var_token, expr]
        elif token == '¬':
            expr = parse_primary()
            return [op_map[token], expr]
        elif token.isalnum():
            left_expr = token
            if tokens and tokens[0] == '(':
                tokens.pop(0)  # Remove '('
                args = []
                while tokens and tokens[0] != ')':
                    args.append(parse_expression())
                    if tokens and tokens[0] == ',':
                        tokens.pop(0)
                tokens.pop(0)  # Remove ')'
                left_expr = [token] + args
            return left_expr
        elif token == '(':
            expr = parse_expression()
            if tokens[0] == ')':
                tokens.pop(0)  # Remove ')'
            return expr
        else:
            return None

    def parse_expression():
        expr = parse_primary()
        while tokens and tokens[0] in precedence:
            op = next_token()
            if precedence[op] == -1:  # Handle quantifiers specially
                continue
            right_expr = parse_primary()
            expr = [op_map[op], expr, right_expr]
        return expr

    parsed_expression = parse_expression()
    return parsed_expression

#### Test Cases

In [61]:
formula = '∀x(p(x)=q(x))'
parsed_formula = parse_logical_expression(formula)
print(parsed_formula)

['forall', 'x', ['equals', ['p', 'x'], ['q', 'x']]]


In [62]:
formula = '∀x (p(f(x)) ↔ q(x))'
parsed_formula = parse_logical_expression(formula)
print(parsed_formula)

['forall', 'x', ['iff', ['p', ['f', 'x']], ['q', 'x']]]


In [63]:
formula = '∀x(p(x)⊃∀y(q(y)⊃r(y)))'
parsed_formula = parse_logical_expression(formula)
print(parsed_formula)

['forall', 'x', ['implies', ['p', 'x'], ['forall', 'y', ['implies', ['q', 'y'], ['r', 'y']]]]]


In [64]:
formula = 'p(a) ∧ q(b)'
parsed_formula = parse_logical_expression(formula)
print(parsed_formula)

['and', ['p', 'a'], ['q', 'b']]


In [65]:
formula = '∀x (p(x) ⊃ q(x))'
parsed_formula = parse_logical_expression(formula)
print(parsed_formula)

['forall', 'x', ['implies', ['p', 'x'], ['q', 'x']]]


### Step-1: Standardize the variables

In [66]:
def standardizeVariables(fol, var_occurrences=None, current_renames=None):
    if var_occurrences is None:
        var_occurrences = {}
    if current_renames is None:
        current_renames = {}

    # Directly return non-list elements.
    if not isinstance(fol, list):
        # Replace the variable with its renamed version if it's been renamed.
        return current_renames.get(fol, fol)

    operator, *args = fol

    if operator in ['forall', 'exists']:
        # For quantifiers, rename the variable if it's already been used.
        quantifier, var, scope = fol
        if var in var_occurrences:
            # Increment and rename the variable.
            new_var_name = f"{var}__{var_occurrences[var]}"
            var_occurrences[var] += 1
        else:
            # Initialize the occurrence counter for new variables.
            var_occurrences[var] = 1
            new_var_name = var

        # Update the renaming for the current scope.
        updated_renames = {**current_renames, var: new_var_name}
        # Recursively standardize the scope with the updated renaming.
        standardized_scope = standardizeVariables(scope, var_occurrences, updated_renames)
        return [quantifier, new_var_name, standardized_scope]
    
    # Recursively apply standardization to the arguments of logical operators.
    return [operator] + [standardizeVariables(arg, var_occurrences, current_renames) for arg in args]

#### Test Cases

In [67]:
fol_formula = ['or', ['forall', 'x', ['P', 'x']], ['exists', 'x', ['Q', 'x']]]
standardized_fol = standardizeVariables(fol_formula)
print(standardized_fol)

['or', ['forall', 'x', ['P', 'x']], ['exists', 'x__1', ['Q', 'x__1']]]


In [68]:
fol_formula = ['forall', 'x', ['exists', 'y', ['or', ['not', ['P', 'x', 'y']], ['exists', 'x', ['Q', 'x', 'z']]]]]
standardized_fol = standardizeVariables(fol_formula)
print(standardized_fol)

['forall', 'x', ['exists', 'y', ['or', ['not', ['P', 'x', 'y']], ['exists', 'x__1', ['Q', 'x__1', 'z']]]]]


In [69]:
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', 'y']]]
standardized_fol = standardizeVariables(fol_formula)
print(standardized_fol)

['forall', 'x', ['exists', 'y', ['P', 'x', 'y']]]


### Step-2: Eliminating the Implication

Helper Function that takes an implication (i.e. the first element should be 'implies') and returns its equivalent form

In [70]:
def eliminateImplication(logic):
    
    result = []
    result.append('or')
    result.append(['not', logic[1]])
    result.append(logic[2])
    
    return result

Helper function that takes an equivalence (i.e. the first element should be 'iff') and returns its equivalent form

In [71]:
def eliminateIFF(logic):
    
    result = []
    result.append('and')
    result.append(eliminateImplication(['implies', logic[1], logic[2]]))
    result.append(eliminateImplication(['implies', logic[2], logic[1]]))
    
    return result

Function that takes a sentence in FOL and return its equivalent form free of implications and equivalences

In [72]:
def parseImplications(logic):
    # If it is an iff statement, replace logic with the one we get by eliminating iffs
    if logic[0] == 'iff' and len(logic) == 3:
        logic = eliminateIFF(logic)
    #If it is an implies statement, replace logic with the one we get by eliminating implies
    if logic[0] == 'implies' and len(logic) == 3:
        logic = eliminateImplication(logic)
    
    #For all the attributes in the logic repeat the process recursively
    for i in range(1, len(logic)):
        # print(logic[i])
        if len(logic[i]) > 1:
            logic[i] = parseImplications(logic[i])

    return logic

#### Test Cases

eliminateImplication

In [73]:
logic = ['implies', ['and', 'A', 'B'], 'C'] # (A ∧ B) ⊃ C
print(eliminateImplication(logic)) # ¬(A ∧ B) ∨ C

['or', ['not', ['and', 'A', 'B']], 'C']


eliminateIFF

In [74]:
logic = ['iff', ['and', 'A', 'B'], 'C'] # (A ∧ B) ↔ C
print(eliminateIFF(logic))

['and', ['or', ['not', ['and', 'A', 'B']], 'C'], ['or', ['not', 'C'], ['and', 'A', 'B']]]


parseImplications

In [75]:
logic = ['or', 'D', ['implies', ['and', 'A', 'B'], 'C']]
print(parseImplications(logic))

['or', 'D', ['or', ['not', ['and', 'A', 'B']], 'C']]


In [76]:
logic = ['implies', ['implies', 'A', 'B'], 'C']
print(parseImplications(logic))

['or', ['not', ['or', ['not', 'A'], 'B']], 'C']


### Step-3: Propagate negation inwards

Helper function that takes a negation (i.e. the first element should be 'not') and returns its equivalent form

In [77]:
def propagateNOT(logic):
    result = []

    #Checking if the inward logic is OR then append AND
    if logic[1][0] == 'or':
        result.append('and')
    #Chicking if inward logic is AND then append OR
    elif logic[1][0] == 'and':
        result.append('or')
    #Checking if inward logic is NOT then return the logic alone
    elif logic[1][0] == 'not':
        return logic[1][1]
    # Checking if the logic is an universal quantifier (∀) then convert to existential (∃) with NOT
    elif logic[1][0] == 'forall':
        # Create a negated existential quantifier with the same variable and expression
        return ['exists', logic[1][1], propagateNOT(['not', logic[1][2]])]
    # Checking if the logic is an existential quantifier (∃) then convert to universal (∀) with NOT
    elif logic[1][0] == 'exists':
        # Create a negated universal quantifier with the same variable and expression
        return ['forall', logic[1][1], propagateNOT(['not', logic[1][2]])]
    else:
        return logic
        
    #For all arguments of the inner list
    for i in range(1, len(logic[1])):
        #check if the first argument is another list
        if len(logic[1][i]) != 1:
            #recursively call to propogate not further inwards
            result.append(propagateNOT(['not', logic[1][i]]))
        else:
            #else append the negation of the single element
            result.append(['not', logic[1][i]])
     
    return result

Function that takes an sentence in FOL and pushes the negation(if any) to the atomic level

In [78]:
def parseNOTs(logic):
    
    #If it is a NOT statement, replace logic with the one we get by propogating not inwards
    if logic[0] == 'not' and len(logic) == 2 and len(logic[1]) != 1:
        logic = propagateNOT(logic)
    
    #For all the attributes in the logic repeat the process recursively
    for i in range(1, len(logic)):
        if len(logic[i]) > 1:
            logic[i] = parseNOTs(logic[i])
    
    #return the final logic statement with NOT propogated inwards
    return logic

#### Test Cases

propagateNOT

In [79]:
logic = ['not', ['P', 'x', 'y']]
print(propagateNOT(logic))

['not', ['P', 'x', 'y']]


In [80]:
logic = ['not', ['and', 'A', 'B']]
print(propagateNOT(logic))

['or', ['not', 'A'], ['not', 'B']]


In [81]:
logic = ['not', ['forall', 'x', ['P', ['f', 'x']]]]
print(propagateNOT(logic))

['exists', 'x', ['not', ['P', ['f', 'x']]]]


In [82]:
logic = ['not', ['forall', 'x', ['P', 'x', 'y']]]
print(propagateNOT(logic))

['exists', 'x', ['not', ['P', 'x', 'y']]]


parseNOTs

In [83]:
logic = ['or', ['not', ['or', ['not', ['P', 'x']], ['P', 'y']]], ['P', 'z']]
print(parseNOTs(logic))

['or', ['and', ['P', 'x'], ['not', ['P', 'y']]], ['P', 'z']]


### Step-4: Eliminating the Existential Quantifier (Skolemization)

For skolemconstants, we will use the format skc0, skc1, skc2, ...

For skolemfunctions, we will use the format skf0, skf1, skf2, ...

Note: ∀x(∃y(P(x,y))) after skolemization is ∀x(P(x, skf0(x)))

Note: Running the same function (without re-running the function def) again on the output of the previous function will give ∀x(P(x, skf1(x)))

In [84]:
def skolemize(fol, universal_vars=[], skolem_counter={'constants': 0, 'functions': 0}):
    if fol[0] == 'forall':
        # Process the formula within the scope of the universal quantifier, adding the variable to the scope
        return ['forall', fol[1]] + [skolemize(fol[2], universal_vars + [fol[1]], skolem_counter)]
    elif fol[0] == 'exists':
        if len(universal_vars) == 0:
            # Replace with a Skolem constant if no universal variables are in scope
            skc = f'skc{skolem_counter["constants"]}'
            skolem_counter["constants"] += 1
            # Replace the existential variable with the Skolem constant in the predicate
            return replace_variable(fol[2], fol[1], skc)
        else:
            # Replace with a Skolem function if universal variables are in scope
            skf = f'skf{skolem_counter["functions"]}(' + ','.join(universal_vars) + ')'
            skolem_counter["functions"] += 1
            return replace_variable(fol[2], fol[1], skf)
    elif isinstance(fol, list):
        # Recursively process each element of the list if it's a complex expression
        return [skolemize(part, universal_vars, skolem_counter) for part in fol]
    else:
        # Return the element unchanged if it's not a quantifier expression
        return fol

def replace_variable(expr, var, replacement):
    if isinstance(expr, str):
        # If the expression is a string, replace the variable if it matches
        return expr.replace(var, replacement)
    elif isinstance(expr, list):
        # If the expression is a list, recursively replace in each element
        return [replace_variable(part, var, replacement) for part in expr]
    else:
        # Return the expression unchanged if it's neither a string nor a list
        return expr

#### Test Cases

In [85]:
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', ['f', 'y']]]]
skolemized_formula = skolemize(fol_formula)
print(skolemized_formula)

['forall', 'x', ['P', 'x', ['f', 'skf0(x)']]]


In [86]:
fol_formula = ['forall', 'x', ['exists', 'y', ['P', 'x', 'y']]]
skolemized_formula = skolemize(fol_formula)
print(skolemized_formula)

['forall', 'x', ['P', 'x', 'skf1(x)']]


### Step-5: Removing the Universal Quantifier

In [87]:
def remove_universal_quantifiers(fol, variables=None):
    if variables is None:
        variables = set()  # Initialize with an empty set for the first call

    if not fol or not isinstance(fol, list):
        return fol, variables  # Return as is if fol is not a list or is empty
    
    if fol[0] == 'forall':
        variables.add(fol[1])  # Add the variable to the set
        # Process the child while passing the variables set for collection
        modified_child, variables = remove_universal_quantifiers(fol[2], variables)
        return modified_child, variables
    else:
        # Initialize an empty list for the processed elements
        processed_elements = []
        # Recursively process each element in the list for nested structures
        for element in fol:
            processed_element, vars_from_element = remove_universal_quantifiers(element, variables)
            variables.update(vars_from_element)  # Merge variables from the current element
            processed_elements.append(processed_element)
        return processed_elements, variables

#### Test Cases

In [88]:
fol_formula = ['forall', 'x', ['and', ['or', 'A', 'B'], ['forall', 'y', ['P', 'x', 'y']]]]
modified_formula, variables = remove_universal_quantifiers(fol_formula)
print(modified_formula)
print("Variables:", variables)

['and', ['or', 'A', 'B'], ['P', 'x', 'y']]
Variables: {'y', 'x'}


### Step-6: Distributing union over intersection

In [89]:
def isDistributionCandidate(logic):
    return logic[0] == 'or' and any(logic[i][0] == 'and' for i in range(1, len(logic)))

def distributeOR(logic):
    result = ['and']
    # Handle distribution for both operands being 'and'.
    if logic[1][0] == 'and' and logic[2][0] == 'and':
        for i in logic[1][1:]:
            for j in logic[2][1:]:
                result.append(['or', i, j])
    # Handle distribution when only one operand is 'and'.
    else:
        and_operand = logic[1] if logic[1][0] == 'and' else logic[2]
        other_operand = logic[2] if logic[1][0] == 'and' else logic[1]
        for i in and_operand[1:]:
            result.append(['or', i, other_operand])
    return result

def parseDistribution(logic):
    if isDistributionCandidate(logic):
        logic = distributeOR(logic)
    for i in range(1, len(logic)):
        if isinstance(logic[i], list) and len(logic[i]) > 1:
            logic[i] = parseDistribution(logic[i])
    return logic

#### Test Cases

In [90]:
logic = ['or', ['and', ['G', 'x', 'y'], ['H', 'z']], ['P', 'x', 'y', 'z']]
print(parseDistribution(logic))

['and', ['or', ['G', 'x', 'y'], ['P', 'x', 'y', 'z']], ['or', ['H', 'z'], ['P', 'x', 'y', 'z']]]


### Step-7: Eliminating the conjunctions and getting the Clauses

In [91]:
def extract_clauses(cnf_formula):
    # Base case
    if len(cnf_formula) == 1:
        return cnf_formula  # Return a single literal in a clause
    
    operator, *operands = cnf_formula
    
    # If the top-level operator is 'and', recursively extract clauses from operands.
    if operator == 'and':
        clauses = []
        for operand in operands:
            clauses.extend(extract_clauses(operand))  # Combine clauses from all 'and' operands
        return clauses
    
    # If the top-level operator is 'or', the whole formula is a single clause.
    elif operator == 'or':
        # Recursively flatten the 'or' structure into a single clause
        return [flatten_or(operands)]
    
    # If the formula does not start with 'and' or 'or', treat it as a single literal clause.
    else:
        return [[cnf_formula]]

def flatten_or(operands):
    flattened_clause = []
    for operand in operands:
        if isinstance(operand, list) and operand[0] == 'or':
            # If the operand is an 'or', recursively flatten and extend the clause
            flattened_clause.extend(flatten_or(operand[1:]))
        else:
            # Otherwise, add the operand (literal or sub-clause) directly to the clause
            flattened_clause.append(operand)
    return flattened_clause

#### Test Cases

In [92]:
cnf_formula = ['and', ['or', ['P', 'x', 'y'], ['Q', 'z']], ['or', ['R', 'x'], ['S', 'y']]]
clauses = extract_clauses(cnf_formula)
print(clauses)

[[['P', 'x', 'y'], ['Q', 'z']], [['R', 'x'], ['S', 'y']]]


In [93]:
cnf_formula = [['P', 'x']]
clauses = extract_clauses(cnf_formula)
print(clauses)

[['P', 'x']]


### Step-8: Renaming the variables for unique variables in each clause

In [94]:
def rename_variables(clauses, variables):
    global_occurrences = {var: 0 for var in variables}
    renamed_clauses = []

    for clause in clauses:
        current_occurrences = {var: 0 for var in variables}
        renamed_clause = []  # For current clause
        
        def rename_var(var):
            # Helper function to rename a variable based on global and current occurrences.
            if var.startswith('skf'):
                # extract the variable name from the skolem function
                var_t = var[var.index('(') + 1 : var.index(')')]
                var_t = var_t.split(',')
                for v in range(len(var_t)):
                    if var_t[v] in variables:
                        if current_occurrences[var_t[v]] == 0:
                            global_occurrences[var_t[v]] += 1
                        current_occurrences[var_t[v]] += 1
                        
                        if global_occurrences[var_t[v]] > 1:
                            var_t[v] = f"{var_t[v]}_{global_occurrences[var_t[v]] - 1}"
                new_var = var[:var.index('(') + 1] + ','.join(var_t) + ')'
                return new_var
            
            if var in variables:
                if current_occurrences[var] == 0:
                    global_occurrences[var] += 1
                current_occurrences[var] += 1
                
                if global_occurrences[var] > 1:
                    return f"{var}_{global_occurrences[var] - 1}"
                return var
            return var

        def rename_var_list(var_list):
            # Helper function
            return [rename_var(var) if isinstance(var, str) else rename_var_list(var) for var in var_list]
        
        for element in clause:
            if isinstance(element, list):
                predicate = element[0]
                args = element[1:]

                renamed_args = []
                for arg in args:
                    if isinstance(arg, list):  # Possible nested function or predicate
                        # Process nested function recursively
                        renamed_args.append([arg[0]] + [rename_var(a) if isinstance(a, str) else rename_var_list(a) for a in arg[1:]])
                    else:
                        renamed_args.append(rename_var(arg))

                if predicate == 'not':
                    renamed_clause.append([predicate, renamed_args[0]])
                else:
                    renamed_clause.append([predicate] + renamed_args)
            else:
                renamed_clause.append(element)

        renamed_clauses.append(renamed_clause)

    return renamed_clauses

#### Test Cases

In [95]:
clauses = [[['G', 'John']], [['not', ['P', 'x']], ['Q', 'y'], ['R', 'x', 'y']], [['not', ['P', 'x']]]]
variables = ['x', 'y', 'z']
renamed_clauses = rename_variables(clauses, variables)
print(renamed_clauses)

[[['G', 'John']], [['not', ['P', 'x']], ['Q', 'y'], ['R', 'x', 'y']], [['not', ['P', 'x_1']]]]


In [96]:
clauses = [[['G', 'John'], ['P', 'skc15']], [['G', 'John'], ['G', 'John']], [['not', ['P', 'x']], ['G', 'John'], ['P', 'skc15']], [['not', ['P', 'x']], ['G', 'John'], ['G', 'John']]]
variables = ['x']
renamed_clauses = rename_variables(clauses, variables)
print(renamed_clauses)

[[['G', 'John'], ['P', 'skc15']], [['G', 'John'], ['G', 'John']], [['not', ['P', 'x']], ['G', 'John'], ['P', 'skc15']], [['not', ['P', 'x_1']], ['G', 'John'], ['G', 'John']]]


### Removing Duplicates

In [97]:
def remove_duplicates(clauses):
    def predicates_are_equal(pred1, pred2):
        if pred1[0] != pred2[0] or len(pred1) != len(pred2):
            return False
        return all(arg1 == arg2 for arg1, arg2 in zip(pred1, pred2))

    def remove_clause_duplicates(clause):
        unique_predicates = []
        for predicate in clause:
            if all(not predicates_are_equal(predicate, unique_pred) for unique_pred in unique_predicates):
                unique_predicates.append(predicate)
        return unique_predicates

    # Apply the removal of duplicates to each clause
    cleaned_clauses = [remove_clause_duplicates(clause) for clause in clauses]
    return cleaned_clauses

#### Test Cases

In [98]:
logic = [[['G', 'John'], ['P', 'skc15']], [['G', 'John'], ['G', 'John']], [['not', ['P', 'x']], ['G', 'John'], ['P', 'skc15']], [['not', ['P', 'x']], ['G', 'John'], ['G', 'John']]]
cleaned_logic = remove_duplicates(logic)
print(cleaned_logic)

[[['G', 'John'], ['P', 'skc15']], [['G', 'John']], [['not', ['P', 'x']], ['G', 'John'], ['P', 'skc15']], [['not', ['P', 'x']], ['G', 'John']]]


### Equality Axioms

Helper function that takes clauses and return the predicates and functions along with their arities

In [99]:
def get_preds_funcs(clauses):
    def get_funcs(function, funcs):
        if not isinstance(function, list):
            return
        else:
            funcs[function[0]] = len(function) - 1
            for i in range(1, len(function)):
                get_funcs(function[i], funcs)
    preds = {}
    funcs = {}
    equals_flag = False
    for clause in clauses:
        for predicate in clause:
            if predicate[0] == 'not' and predicate[1][0] != 'equals':
                preds[predicate[1][0]] = len(predicate[1]) - 1
                for i in range(1, len(predicate[1])):
                    get_funcs(predicate[1][i], funcs)
            elif predicate[0] == 'not' or predicate[0] == 'equals':
                if predicate[0] == 'not':
                    predicate = predicate[1]
                equals_flag = True
                get_funcs(predicate[1], funcs)
                get_funcs(predicate[2], funcs)
            else:
                preds[predicate[0]] = len(predicate) - 1
                for i in range(1, len(predicate)):
                    get_funcs(predicate[i], funcs)

    return preds, funcs, equals_flag

Function that takes the predicates and functions along with their arities(from the helper function) and return the equality axioms

In [100]:
def eq_axioms(preds, funcs):
    new_clauses = []
    # reflexivity
    new_clauses.append([['equals', 'x', 'x']])
    # symmetry
    new_clauses.append([['not', ['equals', 'x', 'y']], ['equals', 'y', 'x']])
    # transitivity
    new_clauses.append([['not', ['equals', 'x', 'y']], ['not', ['equals', 'y', 'z']], ['equals', 'x', 'z']])
    # function equality
    for func in funcs:
        temp = []
        for i in range(funcs[func]):
            temp.append(['not', ['equals', 'x' + str(i+1), 'y' + str(i+1)]])
        temp.append(['equals', [func] + ['x' + str(i+1) for i in range(funcs[func])], [func] + ['y' + str(i+1) for i in range(funcs[func])]])
        new_clauses.append(temp)
    # predicate equality
    for pred in preds:
        temp = []
        for i in range(preds[pred]):
            temp.append(['not', ['equals', 'x' + str(i+1), 'y' + str(i+1)]])
        temp.append(['not', [pred] + ['x' + str(i+1) for i in range(preds[pred])]])
        temp.append([pred] + ['y' + str(i+1) for i in range(preds[pred])])
        new_clauses.append(temp)

    return new_clauses

#### Test Cases

get_preds_funcs

In [101]:
logic = [[['G', ['h', 'x', 'y']], ['equals', ['P', 'y', 'x', ['z', 'y']], ['w', 'm', 'n', 'k']]]]
preds, funcs, equals_flag = get_preds_funcs(logic)
print(preds)
print(funcs)
print(equals_flag)

{'G': 1}
{'h': 2, 'P': 3, 'z': 1, 'w': 3}
True


eq_axioms

In [102]:
preds = {'P': 2, 'Q': 1}
funcs = {'f': 1}
clauses = eq_axioms(preds, funcs)
print(clauses)

[[['equals', 'x', 'x']], [['not', ['equals', 'x', 'y']], ['equals', 'y', 'x']], [['not', ['equals', 'x', 'y']], ['not', ['equals', 'y', 'z']], ['equals', 'x', 'z']], [['not', ['equals', 'x1', 'y1']], ['equals', ['f', 'x1'], ['f', 'y1']]], [['not', ['equals', 'x1', 'y1']], ['not', ['equals', 'x2', 'y2']], ['not', ['P', 'x1', 'x2']], ['P', 'y1', 'y2']], [['not', ['equals', 'x1', 'y1']], ['not', ['Q', 'x1']], ['Q', 'y1']]]


### Combining Steps

In [103]:
def parseLogic(logic):
    #For Empty logic
    if len(logic) == 0:
        return []
    #For logic with one Literal
    if len(logic) == 1:
        return logic
    
    result = standardizeVariables(logic)
    result = parseImplications(logic)
    result = parseNOTs(result)
    result = skolemize(result)
    result, variables = remove_universal_quantifiers(result)
    result = parseDistribution(result)
    result = extract_clauses(result)
    # result = rename_variables(result, list(variables))
    # result = remove_duplicates(result)
    
    return result, variables

#### Test Cases

In [104]:
logic = ['or', ['and', ['G', 'john'], ['forall', 'x', ['implies', ['P', 'x'], ['G', 'john']]]], ['exists', 'y', ['and', ['P', 'y'], ['G', 'john']]]]
print(parseLogic(logic))

([[['G', 'john'], ['P', 'skc0']], [['G', 'john'], ['G', 'john']], [['not', ['P', 'x']], ['G', 'john'], ['P', 'skc0']], [['not', ['P', 'x']], ['G', 'john'], ['G', 'john']]], {'x'})


### Reading Input from File

In [105]:
def read_file(file_name):
    formula = ''

    try:
        with open(file_name, 'r', encoding = 'utf-8') as file:
            formula = str(file.read().strip())  # Read the content and strip any leading/trailing whitespace
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")

    formulas = formula.split("\n")
    # print(formulas)
    return formulas

#### Test Cases

In [106]:
formulas = read_file('formula.txt')
for formula in formulas:
    logic = parse_logical_expression(formula)
    result = parseLogic(logic)
    print(result)

([[['G', 'John'], ['P', 'skc1']], [['G', 'John'], ['G', 'John']], [['not', ['P', 'x']], ['G', 'John'], ['P', 'skc1']], [['not', ['P', 'x']], ['G', 'John'], ['G', 'John']]], {'x'})
([[['J', 'x', 'Pop']]], {'x'})
([[['equals', ['F', 'Pop'], 'C']]], set())


### Converting List Form to Operator Form

In [107]:
def logical_expression(clauses):
    processed_clauses = []

    def format_expression(exp):
        # Handle negation
        if exp[0] == 'not':
            inner_exp = format_expression(exp[1])
            if '=' in inner_exp:
                # Wrap the inner expression in parentheses if it contains an equals sign
                return f'¬({inner_exp})'
            return f'¬{inner_exp}'
        # Handle equals
        elif exp[0] == 'equals':
            left_side = format_literal(exp[1])
            right_side = format_literal(exp[2])
            return f'{left_side} = {right_side}'
        # Handle function calls and literals
        else:
            return format_literal(exp)

    def format_literal(literal):
        if isinstance(literal, list):
            if len(literal) > 1:
                formatted_expression = literal[0] + "(" + ", ".join(map(format_literal, literal[1:])) + ")"
                return formatted_expression
            else:
                return literal[0]  # Single-element list, treated as a simple literal
        else:
            return literal  # Simple string literal

    for clause in clauses:
        temp = " ∨ ".join(format_expression(literal) for literal in clause)
        processed_clauses.append(temp)

    return processed_clauses

#### Test Cases

In [108]:
logic = [[['G', 'John'], ['P', 'skc9']], [['G', 'John'], ['G', 'John']], [['not', ['P', 'x']], ['G', 'John'], ['P', 'skc9']], [['not', ['P', 'x']], ['G', 'John'], ['G', 'John']]]
logical_exp = logical_expression(logic)
for clause in logical_exp:
    print(clause)

G(John) ∨ P(skc9)
G(John) ∨ G(John)
¬P(x) ∨ G(John) ∨ P(skc9)
¬P(x) ∨ G(John) ∨ G(John)


In [109]:
clauses = [[['not', ['P', 'x']], ['Q', 'y']]]
logical_exp = logical_expression(clauses)
for clause in logical_exp:
    print(clause)

¬P(x) ∨ Q(y)


In [110]:
clauses = [[['equals', ['p', 'x'], ['q', 'x']]]]
logical_exp = logical_expression(clauses)
for clause in logical_exp:
    print(clause)

p(x) = q(x)


### Main Function

In [120]:
def main(file_path):
    formulas = read_file(file_path)
    clauses = []
    variables = []
    for formula in formulas:
        logic = parse_logical_expression(formula)
        result, var = parseLogic(logic)
        # print(result)
        clauses.extend(result)
        variables.extend(var)
    preds, funcs, equals_flag = get_preds_funcs(clauses)
    if not funcs:
        max_num = max(preds.values())
    else:
        max_num = max(max(preds.values()), max(funcs.values()))
    variables.append('x')
    variables.append('y')
    for i in range(1, max_num + 1):
        variables.append('x' + str(i))
        variables.append('y' + str(i))
    if equals_flag:
        clauses.extend(eq_axioms(preds, funcs))
    result = rename_variables(clauses, list(set(variables)))
    result = remove_duplicates(result)
    logical_exp = logical_expression(result)
    for clause in logical_exp:
        print(clause)
    return logical_exp, variables

Note: The main function extracts all the clauses from the given input formulas and then renames and removes duplicates from the clauses

In [112]:
clauses, variables = main('formula.txt')

G(John) ∨ P(skc2)
G(John)
¬P(x) ∨ G(John) ∨ P(skc2)
¬P(x_1) ∨ G(John)
J(x_2, Pop)
F(Pop) = C
x_3 = x_3
¬(x_4 = y) ∨ y = x_4
¬(x_5 = y_1) ∨ ¬(y_1 = z) ∨ x_5 = z
¬(x1 = y1) ∨ F(x1) = F(y1)
¬(x1_1 = y1_1) ∨ ¬G(x1_1) ∨ G(y1_1)
¬(x1_2 = y1_2) ∨ ¬P(x1_2) ∨ P(y1_2)
¬(x1_3 = y1_3) ∨ ¬(x2 = y2) ∨ ¬J(x1_3, x2) ∨ J(y1_3, y2)


In [113]:
clauses, varaibles = main('input.txt')

¬M(x) ∨ N(x)
P(x_1, skf2(x_1)) ∨ ¬R(x_1)
Q(skf2(x_2)) ∨ ¬R(x_2)
¬S(f(h(x_3)))
T(f(x_4)) ∨ U(x_4)
P(z)
f(z_1) = g(z_1)
¬(a = b) ∨ V(a)
x_5 = x_5
¬(x_6 = y) ∨ y = x_6
¬(x_7 = y_1) ∨ ¬(y_1 = z_2) ∨ x_7 = z_2
¬(x1 = y1) ∨ f(x1) = f(y1)
¬(x1_1 = y1_1) ∨ h(x1_1) = h(y1_1)
¬(x1_2 = y1_2) ∨ g(x1_2) = g(y1_2)
¬(x1_3 = y1_3) ∨ ¬M(x1_3) ∨ M(y1_3)
¬(x1_4 = y1_4) ∨ ¬N(x1_4) ∨ N(y1_4)
¬(x1_5 = y1_5) ∨ ¬P(x1_5) ∨ P(y1_5)
¬(x1_6 = y1_6) ∨ ¬R(x1_6) ∨ R(y1_6)
¬(x1_7 = y1_7) ∨ ¬Q(x1_7) ∨ Q(y1_7)
¬(x1_8 = y1_8) ∨ ¬S(x1_8) ∨ S(y1_8)
¬(x1_9 = y1_9) ∨ ¬T(x1_9) ∨ T(y1_9)
¬(x1_10 = y1_10) ∨ ¬U(x1_10) ∨ U(y1_10)
¬(x1_11 = y1_11) ∨ ¬V(x1_11) ∨ V(y1_11)


In [122]:
clauses, variables = main('input2.txt')

¬S(x) ∨ ¬W(x) ∨ E(x)
¬W(x_1) ∨ ¬P(x_1) ∨ SP(x_1)
¬R(x_2) ∨ ¬SP(x_2)
¬R(x_3) ∨ We(x_3)
W(x_4)
R(x_5)
S(x_6)


### Putting the clauses in a txt file in the format required by the parser (Done for making the task of the next team easier)

In [ ]:
clauses, base_variables = main('input.txt')

# Function to extract and transform variables based on base variable names
def extract_variables(clause, base_variables):
    # Extract all potential variable occurrences
    potential_vars = re.findall(r"\b([a-z]\d*)(_\d+)?\b", clause)
    actual_vars = {var + (suffix if suffix else '') for var, suffix in potential_vars if var in base_variables}
    return actual_vars

# Function to transform clauses
def transform_clause(clause, variables):
    # Sort variables to ensure proper formatting and predictability
    sorted_vars = sorted(variables)
    if sorted_vars:
        quantified = "∀" + "∀".join(sorted_vars) + " (" + clause + ");"
        return quantified
    else:
        return clause

# Process each clause
transformed_clauses = []
for clause in clauses:
    variables = extract_variables(clause, base_variables)
    transformed_clause = transform_clause(clause, variables)
    transformed_clauses.append(transformed_clause)

with open("output.txt", "w") as file:
    for clause in transformed_clauses:
        file.write(clause + "\n")

print("Transformed clauses have been written to output.txt.")

¬M(x) ∨ N(x)
P(x_1, skf8(x_1)) ∨ ¬R(x_1)
Q(skf8(x_2)) ∨ ¬R(x_2)
¬S(f(h(x_3)))
T(f(x_4)) ∨ U(x_4)
P(z)
f(z_1) = g(z_1)
¬(a = b) ∨ V(a)
x_5 = x_5
¬(x_6 = y) ∨ y = x_6
¬(x_7 = y_1) ∨ ¬(y_1 = z_2) ∨ x_7 = z_2
¬(x1 = y1) ∨ f(x1) = f(y1)
¬(x1_1 = y1_1) ∨ h(x1_1) = h(y1_1)
¬(x1_2 = y1_2) ∨ g(x1_2) = g(y1_2)
¬(x1_3 = y1_3) ∨ ¬M(x1_3) ∨ M(y1_3)
¬(x1_4 = y1_4) ∨ ¬N(x1_4) ∨ N(y1_4)
¬(x1_5 = y1_5) ∨ ¬P(x1_5) ∨ P(y1_5)
¬(x1_6 = y1_6) ∨ ¬R(x1_6) ∨ R(y1_6)
¬(x1_7 = y1_7) ∨ ¬Q(x1_7) ∨ Q(y1_7)
¬(x1_8 = y1_8) ∨ ¬S(x1_8) ∨ S(y1_8)
¬(x1_9 = y1_9) ∨ ¬T(x1_9) ∨ T(y1_9)
¬(x1_10 = y1_10) ∨ ¬U(x1_10) ∨ U(y1_10)
¬(x1_11 = y1_11) ∨ ¬V(x1_11) ∨ V(y1_11)
Transformed clauses have been written to output.txt.
